# 16 Decorators

Comprehensive examples with practical demonstrations.

## Basic Decorator

In [ ]:
# A decorator is a function that takes another function and extends its behavior# without explicitly modifying it.def my_decorator(func):    """Wrapper function that adds behavior before and after the original function"""    def wrapper():        print("Something before the function")        func()  # Call the original function        print("Something after the function")    return wrapper# Apply decorator using @ syntax@my_decoratordef say_hello():    print("Hello!")# Test the decorated functionsay_hello()

## Decorator with Arguments

In [ ]:
# Decorators can accept arguments to control their behaviordef repeat(times):    """Decorator factory that returns a decorator"""    def decorator(func):        def wrapper(*args, **kwargs):            # Execute the function multiple times            for _ in range(times):                result = func(*args, **kwargs)            return result        return wrapper    return decorator@repeat(times=3)def greet(name):    print(f"Hello, {name}!")# This will print the greeting 3 timesgreet("Alice")

## Multiple Decorators (Stacking)

In [ ]:
# You can stack multiple decorators on a single function# They are applied from bottom to top (closest to function first)def uppercase_decorator(func):    def wrapper(*args, **kwargs):        result = func(*args, **kwargs)        return result.upper()    return wrapperdef exclamation_decorator(func):    def wrapper(*args, **kwargs):        result = func(*args, **kwargs)        return result + "!!!"    return wrapper# Applied order: uppercase_decorator -> exclamation_decorator@exclamation_decorator  # Second@uppercase_decorator    # Firstdef get_message(name):    return f"hello, {name}"print(get_message("Bob"))  # Output: HELLO, BOB!!!

## Decorator with functools.wraps

In [ ]:
# functools.wraps preserves the metadata of the original functionfrom functools import wrapsimport timedef timer_decorator(func):    @wraps(func)  # Preserves func.__name__, func.__doc__, etc.    def wrapper(*args, **kwargs):        start = time.time()        result = func(*args, **kwargs)        end = time.time()        print(f"{func.__name__} took {end - start:.4f} seconds")        return result    return wrapper@timer_decoratordef slow_function():    """This function is slow"""    time.sleep(1)    return "Done!"result = slow_function()print(f"Function name: {slow_function.__name__}")print(f"Function docstring: {slow_function.__doc__}")

## Class Decorator

In [ ]:
# Decorators can also be applied to classesdef add_greeting(cls):    """Add a greet method to a class"""    cls.greet = lambda self: f"Hello from {self.name}"    return cls@add_greetingclass Person:    def __init__(self, name):        self.name = nameperson = Person("Charlie")print(person.greet())

## Property Decorator (Getters/Setters)

In [ ]:
# @property decorator creates managed attributes with getter and setter methodsclass Circle:    def __init__(self, radius):        self._radius = radius  # Private attribute        @property    def radius(self):        """Getter for radius"""        return self._radius        @radius.setter    def radius(self, value):        """Setter for radius with validation"""        if value < 0:            raise ValueError("Radius cannot be negative")        self._radius = value        @property    def area(self):        """Calculated property (read-only)"""        return 3.14159 * self._radius ** 2# Usagecircle = Circle(5)print(f"Radius: {circle.radius}")print(f"Area: {circle.area:.2f}")# Update radius using settercircle.radius = 10print(f"New radius: {circle.radius}")print(f"New area: {circle.area:.2f}")

## Practical Example: Logging Decorator

In [ ]:
# Real-world decorator for logging function callsfrom functools import wrapsdef log_function_call(func):    """Log function calls with arguments and return values"""    @wraps(func)    def wrapper(*args, **kwargs):        print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")        result = func(*args, **kwargs)        print(f"[LOG] {func.__name__} returned {result}")        return result    return wrapper@log_function_calldef add(a, b):    return a + b@log_function_calldef multiply(x, y):    return x * yresult1 = add(5, 3)result2 = multiply(4, 7)

## Practical Example: Validation Decorator

In [ ]:
# Decorator for input validationfrom functools import wrapsdef validate_positive(func):    """Ensure the argument is positive"""    @wraps(func)    def wrapper(number):        if number < 0:            raise ValueError("Number must be positive")        return func(number)    return wrapper@validate_positivedef square_root(n):    return n ** 0.5# Test with valid and invalid inputtry:    print(f"Square root of 16: {square_root(16)}")    print(f"Square root of -4: {square_root(-4)}")except ValueError as e:    print(f"Error: {e}")

## Caching Decorator (Memoization)

In [ ]:
# Use lru_cache for automatic memoization to improve performancefrom functools import lru_cache@lru_cache(maxsize=None)  # Cache all resultsdef fibonacci(n):    """Calculate Fibonacci number with caching"""    if n < 2:        return n    return fibonacci(n-1) + fibonacci(n-2)# These calls are fast due to cachingprint(f"Fibonacci(10): {fibonacci(10)}")print(f"Fibonacci(20): {fibonacci(20)}")print(f"Cache info: {fibonacci.cache_info()}")

## Singleton Decorator Pattern

In [ ]:
# Ensure only one instance of a class existsfrom functools import wrapsdef singleton(cls):    """Decorator to make a class a singleton"""    instances = {}        @wraps(cls)    def get_instance(*args, **kwargs):        if cls not in instances:            instances[cls] = cls(*args, **kwargs)        return instances[cls]    return get_instance@singletonclass Database:    def __init__(self):        print("Creating database connection...")        self.connection = "Connected"# Both variables point to the same instancedb1 = Database()db2 = Database()print(f"db1 is db2: {db1 is db2}")  # True - same instance